In [2]:
from matplotlib import pyplot as plt
import numpy as np
import torch as tr
from torch import nn
import pandas as pd
from torch.utils.tensorboard import SummaryWriter
import csv
import draw_rna.draw as draw
from draw_rna.draw_utils import seq2col
import os 
from embeddings import OneHot

In [3]:
secdist = pd.read_csv('RNA_dist/data/SequenceIdentity_f_all.csv')
names=secdist['id']
aux=np.array(secdist.iloc[:,1:])
aux2=secdist.columns[1:]
secdist = pd.DataFrame(aux,index=aux2,columns=aux2)

In [4]:
medioides = pd.read_csv('RNA_dist/medioide.csv')

In [5]:
def file_opener(file_name):
    data_set=[]
    with open(file_name, newline = '') as data:                                                                                         
        data_reader = csv.reader(data, delimiter=',')
        for dat in data_reader:
            data_set.append(dat)
    return data_set
data_set=file_opener('RNA_dist/data/archiveII_220808.csv')

# Dataset

In [6]:
import torch as tr
import torchvision
from torch.utils.data import Dataset, DataLoader
import numpy as np
from torch.nn.functional import cross_entropy

# Metricas

In [7]:
from sklearn.metrics import f1_score
#se suele usar f1 score para todo lo que sean matrices?????????!
def get_f1(ref_batch,predict_batch,L,th=0.5):
    f1 = []
    for ref,predict,l in zip(ref_batch,predict_batch,L):
        ind = tr.where(ref!=-1)
        ref = ref[ind].view(l, l)
        predict = predict[ind].view(l, l)

        #esto si no lo entiendo
        predict = tr.sigmoid(predict) > th

        #parte para tener la matriz triangular superior
        ind = tr.triu_indices(ref.shape[0], ref.shape[1], offset=1)
        ref = ref[ind[0], ind[1]].numpy().ravel()
        predict = predict[ind[0], ind[1]].numpy().ravel()
        f1.append(f1_score(ref, predict, zero_division=1))
    return tr.tensor(f1).mean()

In [8]:
def cluster_score(id,medioides):
    offset_fulldat = tr.empty(len(id),len(medioides))
    cluster_offset=tr.empty(len(medioides))
    
    for i in range(len(id)):
        for j in range(len(medioides)):
            cluster_offset[j]=(secdist[id[i]].loc[medioides.iloc[j][0]])
        offset_fulldat[i]=cluster_offset
    return offset_fulldat

    # for i in range(len(id)):
    #     id_pos=np.where(np.array(secdist['id'])==id[i])[0][0]
    #     for j in range(len(medioides)):
    #         cluster_offset[j]=(secdist.iloc[id_pos][medioides.iloc[j][0]])
    #     offset_fulldat[i]=cluster_offset
    # return offset_fulldat

def concatenate_data (offset_list,sec_data,device):
    return tr.cat((tr.flatten(sec_data,start_dim=1),offset_list.clone().detach().to(device)),dim=1).to(device)

# Modelo

In [14]:
class ignacioDist(nn.Module):
    def __init__(self, embedding_dim=4, device="cpu", negative_weight=.1, lr=1e-3, 
    logger=None, pred_l1=.01, use_scheduler=False,offset=30,max_len=200, **kwargs):
        super().__init__()
        self.device=device
        self.k = offset
        self.len=max_len
        # self.rank = rank
        #por que espesificamente el adam
        #para luego escalar de 0 a 1 el crossentropy
        self.class_weight = tr.tensor([negative_weight, 1.]).float().to(device)
        self.build_graph(embedding_dim,**kwargs)
        self.optim =  tr.optim.Adam(self.parameters(), lr=lr)

        self.to(device)


    
    def build_graph (self,emb_dim,kernel=9,filters=50,rank=32):
        pad = (kernel-1)//2
        dilation = 4
        self.cnn = nn.Sequential(nn.Conv1d(in_channels=emb_dim, out_channels=filters, 
                                        kernel_size=kernel, padding = pad, stride=1),
                                nn.ReLU(),
                                nn.BatchNorm1d(filters),
                                nn.Conv1d(in_channels=filters, out_channels=filters, 
                                        kernel_size=kernel, dilation = dilation,
                                        padding=dilation*pad, stride=1),
                                nn.ReLU(),
                                nn.BatchNorm1d(filters),
                                nn.Conv1d(in_channels=filters, out_channels=filters, 
                                        kernel_size=kernel, dilation = dilation,
                                        padding=dilation*pad, stride=1),
                                nn.ReLU(),
                                nn.BatchNorm1d(filters),
                                nn.Conv1d(in_channels=filters, out_channels=filters//2, 
                                        kernel_size=kernel, padding = pad, stride=1),
                                nn.Sigmoid(),)  
        
        self.convsal1 = nn.Conv1d(in_channels=filters//2, out_channels=rank, kernel_size=kernel, 
                                  padding=pad, stride=1)
        self.convsal2 = nn.Conv1d(in_channels=filters//2,  out_channels=rank, kernel_size=kernel,
                                  padding = pad, stride=1)
        self.linear1a = nn.Sequential(nn.Flatten(),
                                    nn.Sigmoid(),
                                    nn.Linear(in_features=(self.len*rank), out_features=9000,bias=False)
                                    nn.Sigmoid(),
                                    nn.Linear(in_features=90000, out_features=5000,bias=False)
                                    nn.Sigmoid(),
                                    nn.Linear(in_features=5000, out_features=5000,bias=False)
                                    )
        self.linear1b = nn.Sequential(nn.Sigmoid(),
                                    nn.Linear(in_features=1030, out_features=(self.len*rank),bias=False)
                                    )
        self.linear2a = nn.Sequential(nn.Flatten(),
                                    nn.Sigmoid(),
                                    nn.Linear(in_features=(self.len*rank), out_features=1000,bias=False)
                                    )
        self.linear2b = nn.Sequential(nn.Sigmoid(),
                                    nn.Linear(in_features=1030, out_features=(self.len*rank),bias=False)
                                    )
    def forward (self,x,id_list):
        n = x.shape[2]
        y = self.cnn(x)

        offset_list = cluster_score(id_list,medioides)
    
        ya = self.convsal1(y)
        yb = self.convsal2(y)
        
        ya = self.linear1a(ya)
        ya = concatenate_data(offset_list,ya,self.device)
        ya = self.linear1b(ya)
        
        yb = self.linear2a(yb)
        yb = concatenate_data(offset_list,yb,self.device)
        yb = self.linear2b(yb)

        
        ya = ya.view(y.shape[0],32,n)
        yb = yb.view(y.shape[0],32,n)
        
        ya = tr.transpose(ya, -1, -2)

        y = ya @ yb

        yt =  tr.transpose(y, -1, -2)
        
        y = (y+yt)/2

        y = y.view(-1, n, n)
        return y
#loss and optim
    def loss_func(self,yt,y):

        #devuelta lo mismo
        y = y.view(y.shape[0], -1)
        yt = yt.view(yt.shape[0], -1)

        #no es la diferencia?
        pred_l1_loss =  tr.mean(tr.abs(yt[y!=-1])) * 0.01 


        #por que no usar devuelta view
        yt = yt.unsqueeze(1)
        #hizo algo el codigo anterior desp de esto? tipo no se deshizo?
        yt = tr.cat((-yt, yt), dim=1) 
        error = cross_entropy(yt, y, ignore_index=-1, weight=self.class_weight)
        loss= error + pred_l1_loss
        return loss
    
    def tr(self,data_loader):
        self.train()
        avg_loss=0
        avg_f1 = 0
        train_step = 0
        for batch in data_loader:
            x = batch[0].to(self.device)
            y = batch[1].to(self.device)
            id_z = batch[3]

            self.optim.zero_grad()  
            y_tr = self.forward(x,id_z)
        
            loss = self.loss_func(y_tr,y)
            f1 = get_f1(y.cpu(), y_tr.detach().cpu(), batch[2])
            avg_f1 += f1
            avg_loss += loss
            loss.backward() 
            self.optim.step()
            train_step+=1

        avg_f1 /= len(data_loader)
        avg_loss /= len(data_loader)
        return avg_loss,avg_f1
    def tst(self,data_loader):
        self.eval() 
        avg_loss=0
        avg_f1=0
        test_step = 0
        #desactivamos el gradiente por que no hacemos backward()
        with tr.no_grad():       
            for batch in data_loader:
                x = batch[0].to(self.device)
                y = batch[1].to(self.device)
                id_z = batch[3]
                #por que aca y no desp
                y_tst = self.forward(x,id_z)
                # y_tst=tr.tensor([[x], *batch[2:]])
                loss = self.loss_func(y_tst,y)
                #.detach()?????????
                f1 = get_f1(y.cpu(), y_tst.detach().cpu(), batch[2])
                avg_loss += loss
                avg_f1 += f1
                test_step+=1
            avg_loss /= len(data_loader)
            avg_f1 /= len(data_loader)
            
            return avg_loss,avg_f1


# main


In [15]:
import torch as tr
from torch.utils.data import DataLoader, random_split
from dataset import DatasetSeq
maxlen=512
out_path = f"results/"
batch_size = 8
DEVICE = "cuda" 
dataset=DatasetSeq('RNA_dist/data/archiveII_220808.csv',max_len=maxlen)
model = ignacioDist(kernel=11, device=DEVICE,max_len=maxlen,lr=1e-2) 
secdist = pd.read_csv('RNA_dist/data/SequenceIdentity_f_all.csv')
# model.load_state_dict(tr.load(f"{out_path}model_200len.pmt"))
print('entra a hacer las particiones')


n_train = int(0.8 * len(dataset))
n_test = int(0.1 * len(dataset))
n_val = len(dataset) - n_train - n_test



train_data, val_data, test_data = random_split(dataset, [n_train, n_val, n_test], generator=tr.Generator().manual_seed(42))
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

print("train batches", len(train_loader))
print("val batches", len(val_loader))
print("test batches", len(test_loader))
best_f1, patience_counter = 0, 0
for epoch in range(150):
    train_loss,train_f1 = model.tr(train_loader)
    print(f'train loop epoch {epoch}')
    val_loss,val_f1 = model.tst(val_loader)
    if val_f1 > best_f1:
        best_f1 = val_f1
        best_epoch = epoch
        tr.save(model.state_dict(), f"{out_path}model_512len_encoder.pmt")
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter>50:
            break
    print(f"epoch {epoch}: train loss {train_loss:.2f} val loss {val_loss:.2f} train f1 {train_f1:.2f} val f1 {val_f1:.2f}")
print("Start test")
model.load_state_dict(tr.load(f"{out_path}model_512len_encoder.pmt"))
test_loss, test_f1 = model.tst(test_loader)
print(f"best epoch {best_epoch} best_valf1 {best_f1:.3f} test_loss {test_loss:.3f} test_f1 {test_f1:.3f}")

entra a hacer las particiones
train batches 387
val batches 49
test batches 49
train loop epoch 0
epoch 0: train loss 0.23 val loss 0.12 train f1 0.08 val f1 0.08
train loop epoch 1
epoch 1: train loss 0.14 val loss 0.16 train f1 0.08 val f1 0.09
train loop epoch 2
epoch 2: train loss 0.18 val loss 0.15 train f1 0.07 val f1 0.06
train loop epoch 3
epoch 3: train loss 0.18 val loss 0.27 train f1 0.07 val f1 0.07
train loop epoch 4
epoch 4: train loss 0.23 val loss 0.28 train f1 0.06 val f1 0.04
train loop epoch 5
epoch 5: train loss 0.24 val loss 0.17 train f1 0.07 val f1 0.11
train loop epoch 6
epoch 6: train loss 0.26 val loss 0.18 train f1 0.07 val f1 0.09
train loop epoch 7
epoch 7: train loss 0.24 val loss 0.31 train f1 0.07 val f1 0.07
train loop epoch 8
epoch 8: train loss 0.24 val loss 0.71 train f1 0.08 val f1 0.04
train loop epoch 9
epoch 9: train loss 0.33 val loss 0.17 train f1 0.08 val f1 0.10
train loop epoch 10
epoch 10: train loss 0.19 val loss 0.21 train f1 0.09 val f1 

KeyboardInterrupt: 

 nvidia-smi -l 5

In [16]:
print("Start test")
model.load_state_dict(tr.load(f"{out_path}model_512len_OneOfEach.pmt"))
test_loss, test_f1 = model.tst(test_loader)
print(f"best epoch {best_epoch} best_valf1 {best_f1:.3f} test_loss {test_loss:.3f} test_f1 {test_f1:.3f}")

Start test


RuntimeError: Error(s) in loading state_dict for ignacioDist:
	Missing key(s) in state_dict: "linear1a.2.weight", "linear1b.1.weight", "linear2a.2.weight", "linear2b.1.weight". 
	Unexpected key(s) in state_dict: "linear1.2.weight", "linear2.2.weight". 